In [5]:
import datasets
import torch
from collections import OrderedDict
from transformers import BertTokenizer
from transformers import Trainer, TrainingArguments
from network_architecture_v2 import MyBertForSequenceClassification
import argparse
import logging
import numpy as np
logging.disable(logging.WARNING)

# Define a function to compute metrics
def compute_metrics(p):
    logits, labels = p
    predictions = np.argmax(logi        compute_metrics=compute_metrics
ts, axis=-1)
    accuracy = np.mean(predictions == labels)
    return {"accuracy": accuracy}
    
def main():
    # Parse command-line arguments
    qqp_dataset = datasets.load_dataset('glue', 'qqp')

    # Load a pre-trained tokenizer
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)

    # Tokenize the dataset
    def preprocess_function(examples):
        return tokenizer(
            examples['question1'], 
            examples['question2'], 
            truncation='longest_first', 
            padding='max_length', 
            max_length=64,  # Adjust based on your model's max input length
            return_overflowing_tokens=False,  # Suppress the warning
        )

    print('Loading Dataset')
    encoded_dataset = qqp_dataset.map(preprocess_function, batched=True)

    # Load pre-trained model
    model_dicts = torch.load(f'bert-save-1/model_serial_checkpoint_batch_idx=80000')
    new_dict = OrderedDict(model_dicts['model_state_dict'])

    model_serial = torch.load('serialnet_bert_32')
    model_serial.load_state_dict(new_dict)

    # Load a pre-trained model for sequence classification
    model = MyBertForSequenceClassification(model_serial, num_labels=2)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        evaluation_strategy='epoch',     # evaluate each epoch
        learning_rate=1e-5,              # learning rate
        per_device_train_batch_size=32,  # batch size for training
        per_device_eval_batch_size=32,   # batch size for evaluation
        num_train_epochs=5,              # number of training epochs
        weight_decay=0.1,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
        warmup_ratio=.06,
        dataloader_drop_last=True
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=encoded_dataset['train'],         # training dataset
        eval_dataset=encoded_dataset['validation'],     # evaluation dataset
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()

    print(f"Evaluation results: {eval_results}")

if __name__ == "__main__":
    main()


/home/sjiang/braids/pip-test/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading Dataset


Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,0.397000,0.433302,0.787114
2,0.312500,0.349366,0.837465
3,0.293300,0.341155,0.852138
4,0.275800,0.373637,0.855750
5,0.149600,0.422713,0.859759


Evaluation results: {'eval_loss': 0.4227125644683838, 'eval_accuracy': 0.8597585114806018, 'eval_runtime': 97.5322, 'eval_samples_per_second': 414.53, 'eval_steps_per_second': 12.96, 'epoch': 5.0}
